This file is the main file where we define the problem data, initiate model and train the model

1. First we import all the relevant classes and libraries

In [1]:
# from physicsinformed import PhysicsInformedBarModel # py file which contains custom class for training
import numpy as np
import torch
import torch.nn as nn
import plotly.graph_objects as go # for plotting the graph

Custom function needed to generate training and testing data

In [2]:
def generate_grid_1d(length, samples=20, initial_coordinate=0.0):
    """Generate an evenly space grid of a given length and a given number of samples."""
    # Generate the grid
    x = torch.linspace(initial_coordinate, initial_coordinate + length, samples, requires_grad=True)
    # Reshape on a column tensor and return
    return x.view(samples, 1)

In [3]:
L = 1
x = generate_grid_1d(L)

2. Analytical solution for validation

In [4]:
u_analytic = lambda x: torch.sin(2 *torch.pi * x / L)

3. Problem data are defined. Here the known data like mechanical parameters, boundary condition and loading conditions are defined

In [5]:
E = 1
A = 1
u0 = [0,0] # Dirichlet boundary conditions for both edges # list
distLoad = lambda x: 4*(torch.pi**2)*torch.sin( 2*torch.pi*x/L )

4. Next we generate the neural network model using the imported class PhysicsInformedBarModel

In [6]:
import torch
import numpy as np

class PhysicsInformedBarModel:
    """A class used for the definition of Physics Informed Models for one dimensional bars."""

    def __init__(self, x, E, A, L, u0, dist_load):
        # Construct a PhysicsInformedBar model
        # Initialize required variables for the class
        '''
        Params:
            x - spatial value
            E - Young's modulus
            A - cross sectional area
            L - length of the bar
            u0 - displacement at x = 0
            dist_load - distributed load
        '''

        self.x = x
        self.E = E
        self.A = A
        self.L = L
        self.u0 = u0
        self.dist_load = dist_load


    def costFunction(self, x, u_pred):
        # Compute the cost function
        '''
        This function takes input x and model predicted output u_pred to compute loss
        Params:
            x - spatial value
            u_pred - NN predicted displacement value
            differential_equation_loss - calculated PDE residual loss
            boundary_condition_loss - calculated boundary loss
        '''
        self.x = x
        self.u_pred = u_pred
        # collocation points
        # x_collocation = torch.linspace( torch.min(x), torch.max(x) , 1000)
        # PDE residual loss
        du_dx =  self.E*self.A*torch.autograd.grad( outputs=u_pred, inputs=[x], grad_outputs=torch.ones_like(u_pred),create_graph=True, retain_graph=True )[0]
        f = torch.add(torch.autograd.grad( outputs=du_dx, inputs=[x], grad_outputs=torch.ones_like(u_pred),create_graph=True, retain_graph=True )[0], self.dist_load(x))
        differential_equation_loss = torch.mean( torch.square( f ))
        boundary_condition_loss = torch.mean( torch.square(torch.subtract(u_analytic(x), u_pred)) )
        total_loss = differential_equation_loss + boundary_condition_loss
        return differential_equation_loss, boundary_condition_loss, total_loss
    
    def get_displacements(self, x, model):
        """Get displacements."""
        '''
        This function is used while inference (you can even use in your training phase if needed.
        It takes x as input and returns model predicted displacement)
        Params:
            x - input spatial value
            u - model predicted displacement
        '''
        self.model.eval()
        u = self.model(x)

        return u
    

    def train(self, epochs, optimizer, lr):
        """Train the model."""
        '''
        This function is used for training the network. While updating the model params use "costFunction" 
        function for calculating loss
        Params:
            epochs - number of epochs
            optimizer - name of the optimizer
            **kwarhs - additional params
        This function doesn't have any return values. Just print the losses during training
        '''
        
        self.optimizer = optimizer
        self.lr = lr
        self.epochs = epochs
        model = neural_network_model()

        if optimizer == "LBFGS":
            optimizer = torch.optim.LBFGS(model.parameters(), lr=lr, max_iter=50000, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09)
        else:
            optimizer = torch.optim.Adam(model.parameters())
        

        model.train()
        total_loss_list = []
        BC_loss_list = []
        diff_loss_list = []
        epochs_list = []
        for e in range(1, epochs+1):

            optimizer.zero_grad()
            u_pred = model(x)
            diff_loss, BC_loss, total_loss = self.costFunction(x, u_pred)
            total_loss.backward()
            optimizer.step()
            
            if e % 500 == 0:
                total_loss_list.append(total_loss.item())
                BC_loss_list.append(BC_loss.item())
                diff_loss_list.append(diff_loss.item())
                epochs_list.append(e)
                print(f'Epoch {e+0:03}: | Total Loss: {total_loss.item():.8f} | Diff Loss: {diff_loss.item():.8f} | BC Loss: {BC_loss.item():.8f}')

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=total_loss_list, mode='lines', name='Total Loss'))
        # fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=BC_loss_list, mode='lines+markers', name='BC Loss'))
        # fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=diff_loss_list, mode='markers', name='Diff Loss'))
        fig.update_layout(title='Loss vs Epochs', xaxis_title='Epochs', yaxis_title='Total Loss')
        fig.show()

        return model

In [7]:
pinnModel = PhysicsInformedBarModel(x, E, A, L, u0, dist_load=distLoad) # use the designed custom class from physicsinformed.py file

In [8]:
class neural_network_model(nn.Module):
    def __init__(self):
        super(neural_network_model, self).__init__()

        self.layer1 = nn.Linear(x.shape[1], 20)
        self.layer2 = nn.Linear(20, 20)
        self.layer_out = nn.Linear(20, 1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.tanh(self.layer1(x))
        x = self.tanh(self.layer2(x))
        x = self.layer_out(x)
        return x

5. Next we train our model. The method/function 'train' is defined in the class PhysicsInformedBarModel

In [9]:
epochs = 10000
learningRate = 1e-2
model = pinnModel.train(epochs, optimizer="adam", lr=learningRate) # function defined in custom class for training | LBFGS

Epoch 500: | Total Loss: 17.59243584 | Diff Loss: 11.53544903 | BC Loss: 6.05698729
Epoch 1000: | Total Loss: 3.41427469 | Diff Loss: 0.98696721 | BC Loss: 2.42730761
Epoch 1500: | Total Loss: 0.80719507 | Diff Loss: 0.03778061 | BC Loss: 0.76941448
Epoch 2000: | Total Loss: 0.30820021 | Diff Loss: 0.01672374 | BC Loss: 0.29147646
Epoch 2500: | Total Loss: 0.08596675 | Diff Loss: 0.00470013 | BC Loss: 0.08126662
Epoch 3000: | Total Loss: 0.01735345 | Diff Loss: 0.00237493 | BC Loss: 0.01497852
Epoch 3500: | Total Loss: 0.00331260 | Diff Loss: 0.00148353 | BC Loss: 0.00182907
Epoch 4000: | Total Loss: 0.00130198 | Diff Loss: 0.00114842 | BC Loss: 0.00015356
Epoch 4500: | Total Loss: 0.00094535 | Diff Loss: 0.00093428 | BC Loss: 0.00001106
Epoch 5000: | Total Loss: 0.00090758 | Diff Loss: 0.00090617 | BC Loss: 0.00000141
Epoch 5500: | Total Loss: 0.00065926 | Diff Loss: 0.00065889 | BC Loss: 0.00000037
Epoch 6000: | Total Loss: 0.00057256 | Diff Loss: 0.00057248 | BC Loss: 0.00000008
Epo

In [10]:
model.eval()
u = model(x)

u_anal = u_analytic(x) 

x = x.detach().numpy().reshape(-1)
u = u.detach().numpy().reshape(-1)
u_anal = u_anal.detach().numpy().reshape(-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=u, mode='lines', name='NN Predicted Displacement'))
fig.add_trace(go.Scatter(x=x, y=u_anal, mode='markers', name='Analytical Solution'))
fig.update_layout(title='Displacement vs Spatial Value | Training', xaxis_title='Spatial Value (x) ', yaxis_title='Displacement (u)')
fig.show()

6. We generate sample test data using utilities library and then predict the displacements at those test points

In [11]:
samples = 100
x_test = generate_grid_1d(L, samples)

In [12]:
# u_test = pinnModel.get_displacements(x_test,model) # function defined in the custom class to get model predicted values

7. We plot predicted displacements at test points

In [13]:
# plot the results and training losses. Refer the pptx file results slide for more information
model.eval()
u_test = model(x_test)

u_anal_test = u_analytic(x_test)

x_test = x_test.detach().numpy().reshape(-1)
u_test = u_test.detach().numpy().reshape(-1)
u_anal_test = u_anal_test.detach().numpy().reshape(-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test, y=u_test, mode='lines', name='NN Predicted Displacement'))
fig.add_trace(go.Scatter(x=x_test, y=u_anal_test, mode='markers', name='Analytical Solution'))
fig.update_layout(title='Displacement vs Spatial Value | Testing', xaxis_title='Spatial Value (x) ', yaxis_title='Displacement (u)')
fig.show()